# Import Dataset


In [2]:
%run -i "CleanedData.py"

/workspaces/Dq_Datascientist/3 - Data_Cleaning/03_Data_Cleaning_Walktrough/CleanedData.py:67: SyntaxWarning: invalid escape sequence '\('
  y = re.findall("\(.+\)", x)
/workspaces/Dq_Datascientist/3 - Data_Cleaning/03_Data_Cleaning_Walktrough/CleanedData.py:75: SyntaxWarning: invalid escape sequence '\('
  y = re.findall("\(.+\)", x)
/workspaces/Dq_Datascientist/3 - Data_Cleaning/03_Data_Cleaning_Walktrough/CleanedData.py:138: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  combined['school_dist'] = combined['DBN'].apply(lambda x:x[0:2])


In [3]:
combined.head()

,DBN,SCHOOL NAME_x,Num of SAT Test Takers,SAT Critical Reading Avg. Score,SAT Math Avg. Score,SAT Writing Avg. Score,sat_score,SchoolName,AP Test Takers,Total Exams Taken,...,Location 1,Community Board,Council District,Census Tract,BIN,BBL,NTA,lon,lat,school_dist
0,01M292,HENRY STREET SCHOOL FOR INTERNATIONAL STUDIES,29,355.0,404.0,363.0,1122.0,0,0.0,0.0,...,"220 Henry Street\nNew York, NY 10002\n(40.7137...",3.0,1.0,201.0,1003223.0,1.002690e+09,Lower East Side ...,-73.98526,40.713764,01
1,01M292,HENRY STREET SCHOOL FOR INTERNATIONAL STUDIES,29,355.0,404.0,363.0,1122.0,0,0.0,0.0,...,"220 Henry Street\nNew York, NY 10002\n(40.7137...",3.0,1.0,201.0,1003223.0,1.002690e+09,Lower East Side ...,-73.98526,40.713764,01
2,01M292,HENRY STREET SCHOOL FOR INTERNATIONAL STUDIES,29,355.0,404.0,363.0,1122.0,0,0.0,0.0,...,"220 Henry Street\nNew York, NY 10002\n(40.7137...",3.0,1.0,201.0,1003223.0,1.002690e+09,Lower East Side ...,-73.98526,40.713764,01
3,01M292,HENRY STREET SCHOOL FOR INTERNATIONAL STUDIES,29,355.0,404.0,363.0,1122.0,0,0.0,0.0,...,"220 Henry Street\nNew York, NY 10002\n(40.7137...",3.0,1.0,201.0,1003223.0,1.002690e+09,Lower East Side ...,-73.98526,40.713764,01
4,01M292,HENRY STREET SCHOOL FOR INTERNATIONAL STUDIES,29,355.0,404.0,363.0,1122.0,0,0.0,0.0,...,"220 Henry Street\nNew York, NY 10002\n(40.7137...",3.0,1.0,201.0,1003223.0,1.002690e+09,Lower East Side ...,-73.98526,40.713764,01


# 1) Introduction

we began investigating possible relationships between SAT scores and demographics. In order to do this, we acquired several datasets containing information about **New York City public schools**. We **cleaned** them, then **combined** them into a single dataset named combined that we're now ready to analyze and visualize.

In this lesson, we'll discover correlations, create plots, and then make maps. The first thing we'll do is find any **correlations** between `columns` and `sat_score`.


# 2 - 3) Finding Correlations With the r Value

We'll be using the **r value**, also called Pearson's correlation coefficient, to measure how closely two sequences of numbers are correlated.

An **r value** falls between `-1` and `1`. The value determines whether two columns are:

* positively correlated;
* not correlated;
* or negatively correlated.

The closer to `1` the **r value** is, the stronger the positive correlation between the two columns. The closer to `-1` the **r value** is, the stronger the negative correlation (i.e., the more "opposite" the columns are). The closer to `0`, the weaker the correlation.

In general, r values above `.25` or below `-.25` are enough to qualify a correlation as interesting.

We can use the pandas `pandas.DataFrame.corr()` method to find correlations between columns in a dataframe. The method returns a new dataframe where the index for each column and row is the name of a column in the original dataset.

# Instructions

1. Use the `pandas.DataFrame.corr()` method on the combined dataframe to find all possible correlations. Assign the result to correlations.
    * Ensure you use the `numeric_only=True` argument to only return the correlations between numeric columns.

1. Filter correlations so that it only shows correlations for the column `sat_score`.
1. Display all of the rows in correlations and examine them.

In [3]:
correlations = combined.corr(numeric_only=True)['sat_score']
print(correlations)

SAT Critical Reading Avg. Score    0.995822
SAT Math Avg. Score                0.990895
SAT Writing Avg. Score             0.995898
sat_score                          1.000000
AP Test Takers                     0.392718
                                     ...   
Census Tract                       0.060592
BIN                                0.069487
BBL                                0.072702
lon                               -0.057324
lat                               -0.072652
Name: sat_score, Length: 74, dtype: float64
